In [1]:
# import libraries
from pyspark.context import SparkContext,SparkConf
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Assignments

#### 1)What are the lowest and highest temperatures measured each year for the period 1950-2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the temperature-readings.csv file.¶

In [2]:
temp_csv = spark.read.csv("temperature-readings-tiny.csv", header=False,sep=";")
df=temp_csv.selectExpr("_c0 as Station_number", "_c1 as Date", "_c2 as Time", "_c3 as Temperature","_c4 as Quality")
df=df.withColumn('Date',F.to_date("Date", "yyyy-MM-dd"))
df=df.orderBy(df.Temperature,ascending=False)
df.show()
df.printSchema()

+--------------+----------+--------+-----------+-------+
|Station_number|      Date|    Time|Temperature|Quality|
+--------------+----------+--------+-----------+-------+
|        102170|2016-06-11|06:00:00|        9.9|      G|
|        102190|1955-09-15|12:00:00|        9.9|      Y|
|        102190|1957-05-02|12:00:00|        9.9|      Y|
|        102190|1956-09-12|00:00:00|        9.9|      Y|
|        102190|1957-06-23|12:00:00|        9.9|      Y|
|        102190|1960-06-17|06:00:00|        9.9|      Y|
|        102170|2014-09-21|18:00:00|        9.9|      G|
|        102170|2015-05-26|18:00:00|        9.9|      G|
|        102170|2014-10-08|06:00:00|        9.9|      G|
|        102170|2015-06-17|06:00:00|        9.9|      G|
|        102190|1956-08-25|00:00:00|        9.9|      Y|
|        102190|1956-09-05|00:00:00|        9.9|      Y|
|        102170|2015-06-18|06:00:00|        9.9|      G|
|        102190|1956-06-16|12:00:00|        9.8|      Y|
|        102190|1955-09-16|06:0

In [3]:
# maximun temperature per year
df.createOrReplaceTempView("temps_table")
query_max_temps="SELECT YEAR(Date) as year, MAX(FLOAT(Temperature)) AS max_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ORDER BY max_Temperature DESC"

max_temps=spark.sql(query_max_temps)
max_temps.show()
max_temps.printSchema()

+----+---------------+
|year|max_Temperature|
+----+---------------+
|2014|           29.1|
|1960|           29.0|
|1959|           28.2|
|1958|           28.1|
|1956|           26.0|
|1957|           25.2|
|1955|           20.4|
|1961|           19.0|
|2013|           10.2|
+----+---------------+

root
 |-- year: integer (nullable = true)
 |-- max_Temperature: float (nullable = true)



In [4]:
# min temperatures per year
query_min_temps="SELECT YEAR(Date) as year, MIN(FLOAT(Temperature)) AS min_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ORDER BY min_Temperature ASC"

min_temps=spark.sql(query_min_temps)
min_temps.show()
min_temps.printSchema()

+----+---------------+
|year|min_Temperature|
+----+---------------+
|1956|          -30.0|
|1960|          -28.3|
|1958|          -27.9|
|1955|          -26.2|
|2014|          -24.3|
|1961|          -23.5|
|1959|          -23.2|
|1957|          -19.9|
|2013|          -13.3|
+----+---------------+

root
 |-- year: integer (nullable = true)
 |-- min_Temperature: float (nullable = true)



### Question 1.a

In [5]:
# max temperature for every station per year
query_max_temps_station="SELECT YEAR(Date) as year, Station_number,MAX(FLOAT(Temperature)) AS max_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ,Station_number ORDER BY max_Temperature DESC"

max_temps_station=spark.sql(query_max_temps_station)
max_temps_station.show()

+----+--------------+---------------+
|year|Station_number|max_Temperature|
+----+--------------+---------------+
|2014|        102170|           29.1|
|1960|        102190|           29.0|
|1959|        102190|           28.2|
|1958|        102190|           28.1|
|1956|        102190|           26.0|
|1957|        102190|           25.2|
|1955|        102190|           20.4|
|1961|        102190|           19.0|
|2013|        102170|           10.2|
+----+--------------+---------------+



In [6]:
# min temperature for every station per year
query_min_temps_station="SELECT YEAR(Date) as year, Station_number,MIN(FLOAT(Temperature)) AS min_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ,Station_number ORDER BY min_Temperature ASC"

min_temps_station=spark.sql(query_min_temps_station)
min_temps_station.show()

+----+--------------+---------------+
|year|Station_number|min_Temperature|
+----+--------------+---------------+
|1956|        102190|          -30.0|
|1960|        102190|          -28.3|
|1958|        102190|          -27.9|
|1955|        102190|          -26.2|
|2014|        102170|          -24.3|
|1961|        102190|          -23.5|
|1959|        102190|          -23.2|
|1957|        102190|          -19.9|
|2013|        102170|          -13.3|
+----+--------------+---------------+



### Question 1.b

In [7]:
# see Lab 2 ===> solution with pandas

### Question 2

#### 2)Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees.Repeat the exercise,this time taking only distinct readings from each station. That is, if a station reported are a ding above 10 degrees in some month,then it appears only once in the count for that month.In this exercise you will use the temperature-readings.csvfile.¶
#### The out put should contain the following information:Year, month, count

In [8]:
# using the temps_table from before

query_over10=" SELECT YEAR(Date) AS year , MONTH(Date) AS month ,COUNT(Temperature) AS count \
FROM temps_table WHERE FLOAT(Temperature) >=10 AND YEAR(Date) >=1950 AND YEAR(Date) <= 2014  GROUP BY year,month ORDER BY year,month ASC"

spark.sql(query_over10).show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|1955|    9|   55|
|1955|   10|   23|
|1956|    3|    4|
|1956|    4|    7|
|1956|    5|   60|
|1956|    6|   92|
|1956|    7|  108|
|1956|    8|   84|
|1956|    9|   54|
|1956|   10|   17|
|1957|    3|    1|
|1957|    4|   13|
|1957|    5|   46|
|1957|    6|   72|
|1957|    7|  109|
|1957|    8|   96|
|1957|    9|   43|
|1957|   10|    9|
|1958|    4|    4|
|1958|    5|   40|
+----+-----+-----+
only showing top 20 rows



In [9]:
# now using distinct temperatures

query_over10_distinct=" SELECT YEAR(Date) AS year , MONTH(Date) AS month ,AVG(DISTINCT(Temperature)) AS count \
FROM temps_table WHERE FLOAT(Temperature) >=10 AND YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year,month ORDER BY year,month ASC"

spark.sql(query_over10_distinct).show()

+----+-----+------------------+
|year|month|             count|
+----+-----+------------------+
|1955|    9| 14.57714285714286|
|1955|   10|12.440000000000001|
|1956|    3|             12.05|
|1956|    4|             11.72|
|1956|    5| 14.83181818181818|
|1956|    6|15.277586206896546|
|1956|    7|17.165789473684207|
|1956|    8|13.993877551020407|
|1956|    9|              13.1|
|1956|   10|11.753846153846153|
|1957|    3|              10.6|
|1957|    4|12.246153846153847|
|1957|    5| 14.45151515151515|
|1957|    6|16.168627450980395|
|1957|    7|17.081159420289858|
|1957|    8|              15.9|
|1957|    9|12.411111111111113|
|1957|   10|            11.475|
|1958|    4|              12.1|
|1958|    5|13.406451612903224|
+----+-----+------------------+
only showing top 20 rows



### Question 3

#### 3)Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe.In this exercise you will use the temperature-readings.csvfile.¶
#### The output should contain the following information: Year, month, station number, average monthly temperature


### The monthly average is calculated by averaging the daily maximums and minimums for that month.For example, to get the monthly average for October, take maximums and minimums for each day, sum them up and divide by 62 (which is the same as taking the daily averages, summing them up and divide by the number of days).

In [10]:
query=" WITH sub AS (SELECT YEAR(Date) AS year , MONTH(Date) AS month , DAY(Date) AS day,Station_number AS station,AVG(FLOAT(Temperature)) AS avg_daily \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year,month,day,station ORDER BY (year,month,day) DESC) \
SELECT year,month,station,AVG(avg_daily) AS avg_monthly FROM sub GROUP BY year,month,station \
ORDER BY (year,month,station) DESC"

spark.sql(query).show()



+----+-----+-------+--------------------+
|year|month|station|         avg_monthly|
+----+-----+-------+--------------------+
|2014|   12| 102170|  -5.801612894020734|
|2014|   11| 102170|  2.5249999977648256|
|2014|   10| 102170|   7.106451625304837|
|2014|    9| 102170|   8.584999987483025|
|2014|    8| 102170|   13.86935483255694|
|2014|    7| 102170|    19.6596774132021|
|2014|    6| 102170|  14.443333387374878|
|2014|    5| 102170|  10.756451619728919|
|2014|    4| 102170|   4.776666717727979|
|2014|    3| 102170|  1.8967741969371996|
|2014|    2| 102170|  0.6749999944918922|
|2014|    1| 102170|  -4.106451601751389|
|2013|   12| 102170|  0.7096774189943268|
|2013|   11| 102170|-0.05166668320695559|
|1961|    5| 102190|  10.615333366394044|
|1961|    4| 102190|   4.507777822679944|
|1961|    3| 102190|  2.7086021617375398|
|1961|    2| 102190|  -2.133333349955224|
|1961|    1| 102190|  -7.215053754468119|
|1960|   12| 102190| -2.4274193498115704|
+----+-----+-------+--------------

#### 4) Provide a list of stations with their associated maximum measured temperatures and maximum measured daily precipitation. Show only those stations where the maximum temperature is between 25 and 30 degrees and maximum daily precipitation is between 100 mm and 200mm.In this exercise you will use the temperature-readings.csvand precipitation-readings.csvfiles.The output should contain the following information: Station number, maximum measured temperature, maximum daily precipitation¶
#### HINT: The correct result for this question should be empty


In [11]:
# using temps_table from above
# make a table for the presipitation
precip_csv=spark.read.csv("C:/Users/quartermaine/Documents/Big Data Analytics Labs/precipitation-readings.csv", header=False,sep=";")
dt=precip_csv.selectExpr("_c0 as Station_number", "_c1 as Date", "_c2 as Time", "_c3 as Precipitation","_c4 as Quality")
dt=dt.withColumn('Date',F.to_date("Date", "yyyy-MM-dd"))
dt=dt.orderBy(dt.Precipitation,ascending=False)
dt.show()
dt.printSchema()

+--------------+----------+--------+-------------+-------+
|Station_number|      Date|    Time|Precipitation|Quality|
+--------------+----------+--------+-------------+-------+
|         94390|1998-08-13|11:00:00|          9.9|      Y|
|         93520|2008-08-03|14:00:00|          9.9|      Y|
|         97280|2006-08-01|04:00:00|          9.9|      Y|
|         93520|1996-05-15|00:00:00|          9.9|      Y|
|         96040|2002-06-24|12:00:00|          9.9|      Y|
|        173900|2003-05-14|19:00:00|          9.9|      Y|
|         97280|2001-08-08|23:00:00|          9.9|      Y|
|        167990|2014-07-16|18:00:00|          9.9|      G|
|        172940|2016-06-13|17:00:00|          9.9|      G|
|         93520|2008-08-03|13:00:00|          9.9|      Y|
|         99280|2013-09-17|20:00:00|          9.9|      G|
|        117430|2014-08-18|01:00:00|          9.9|      G|
|        103100|2002-06-11|15:00:00|          9.9|      Y|
|        106160|2011-06-23|12:00:00|          9.9|      

In [12]:
dt.createOrReplaceTempView("precips_table")

In [13]:
query_temp_precip="WITH sub AS (SELECT t.Station_number,t.Date,t.Temperature,p.Precipitation \
FROM temps_table AS t , precips_table AS p WHERE p.Station_number=t.Station_number) \
SELECT Station_number , MAX(FLOAT(Precipitation)) AS Max_Precip,MAX(FLOAT(Temperature)) AS Max_Temp \
FROM sub  GROUP BY Station_number,YEAR(Date),MONTH(date),DAY(Date) HAVING MAX(FLOAT(Temperature)) BETWEEN 20 AND 30 \
AND MAX(FLOAT(Precipitation)) BETWEEN 10 AND 20"

spark.sql(query_temp_precip).show()

+--------------+----------+--------+
|Station_number|Max_Precip|Max_Temp|
+--------------+----------+--------+
+--------------+----------+--------+



#### 5)Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file)for the period1993-2016. In order to do this, you will first need to calculate the total monthly precipitation for each station before calculating the monthly average (by averaging over stations).In this exercise you will use the precipitation-readings.csvand stations-Ostergotland.csv files. HINT(not for the SparkSQL lab): Avoid using joins here! stations-Ostergotland.csv is small and if distributed will cause a number of unnecessary shuffles when joined with precipitationRDD. If you distribute precipitation-readings.csv then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.¶
#### The output should contain the following information: Year,month,average monthly precipitation

In [14]:
# using the precips_table from above 
# make table for Ostegorland
ost_csv=spark.read.csv("C:/Users/quartermaine/Documents/Big Data Analytics Labs/stations-Ostergotland.csv", header=False,sep=";")
DF=ost_csv.selectExpr("_c0 as Station_number", "_c1 as Station_name ","_c2 as Measurment_height", 
                         "_c3 as Latitude","_c4 as Longtitude","_c5 as Readings_from","_c6 as Redings_to","_c7 as Elevation")

DF.show()
DF.printSchema()


+--------------+--------------------+-----------------+--------+----------+-------------------+-------------------+---------+
|Station_number|        Station_name|Measurment_height|Latitude|Longtitude|      Readings_from|         Redings_to|Elevation|
+--------------+--------------------+-----------------+--------+----------+-------------------+-------------------+---------+
|         84260|             Örberga|              2.0| 58.4274|    14.826|1990-11-01 00:00:00|1993-03-31 23:59:59|    105.0|
|         85630|           Västra Ny|              2.0|   58.65|     15.05|1961-01-01 00:00:00|1966-12-31 23:59:59|    120.0|
|         85460|         Kettstaka A|              2.0| 58.7165|     15.03|1995-08-01 00:00:00|2016-10-01 08:00:00|    225.0|
|         85450|            Godegård|              2.0| 58.7936|   15.1694|1961-01-01 00:00:00|1988-02-29 23:59:59|    130.0|
|         85490|          Zinkgruvan|              2.0| 58.8063|   15.1285|1983-09-01 00:00:00|2016-08-31 23:59:59|   

In [15]:
DF.createOrReplaceTempView("ost_table")


In [16]:
# I didn't use the approach we use in the 3rd question but the classical definition!!!!!

query_max_precip="WITH sub AS(SELECT Station_number,YEAR(Date) AS year,MONTH(Date) AS month ,DAY(Date) AS day,Precipitation \
FROM precips_table  WHERE Station_number IN (SELECT Station_number FROM ost_table) ) \
SELECT year, month,AVG(FLOAT(Precipitation)) AS avg_precipitation FROM sub \
WHERE INT(year) BETWEEN 1993 AND 2016 GROUP BY year,month ORDER BY year ASC,month ASC"

spark.sql(query_max_precip).show()

+----+-----+--------------------+
|year|month|   avg_precipitation|
+----+-----+--------------------+
|1993|    4|                 0.0|
|1993|    5|0.028513513565868946|
|1993|    6| 0.09323432421920323|
|1993|    7| 0.12874494014400864|
|1993|    8| 0.11414427218545108|
|1993|    9| 0.06023738923474306|
|1993|   10| 0.05814266531736982|
|1993|   11| 0.06053748258944111|
|1993|   12| 0.05167130978575656|
|1994|    1| 0.04062500083819032|
|1994|    2|0.033936651943316104|
|1994|    3|  0.0516483520802397|
|1994|    4|0.032719547039810726|
|1994|    5|  0.0349582173967926|
|1994|    6| 0.06442857197352818|
|1994|    7|                 0.0|
|1994|    8| 0.08155339828145984|
|1994|    9| 0.13611510657578063|
|1994|   10| 0.04486486480848209|
|1994|   11|0.021835883330652214|
+----+-----+--------------------+
only showing top 20 rows



In [72]:
#calculate the avg precipication like the temperature in question 3 

In [73]:
q2="WITH sub AS( SELECT YEAR(Date) AS year,MONTH(Date) AS month ,DAY(Date) AS day,Station_number,Precipitation \
FROM precips_table WHERE Station_number IN (SELECT Station_number FROM ost_table) ) \
SELECT year,month,day,AVG(FLOAT(Precipitation)) AS avg_daily FROM sub \
WHERE INT(year) BETWEEN 1993 AND 2016 GROUP BY year,month,day"

T2=spark.sql(q2)
T2.show()
T2.printSchema()

+----+-----+---+--------------------+
|year|month|day|           avg_daily|
+----+-----+---+--------------------+
|2007|    9| 22| 0.17222222064932188|
|2011|    8| 10| 0.08333333405769533|
|1999|    9| 20|0.020567376562889587|
|2006|    8| 23| 0.08661971902343589|
|2002|    6| 30| 0.07014925422063514|
|2005|   12|  1|  0.0729166663562258|
|1999|   12| 31| 0.10559440633425346|
|2007|    1| 18|  0.0708333344405724|
|2002|   12| 28| 0.18333333606521288|
|2015|   10| 22|0.021142857117312296|
|1996|   11| 28| 0.19014084900558834|
|1997|    5| 13|0.022916666852931183|
|1999|    6| 25| 0.04236111158712043|
|2014|    2| 10|  0.0467741939649787|
|1996|    6|  1| 0.03623188454387845|
|1993|    8|  9| 0.07916666784634192|
|2006|    2|  3|0.013888888940629032|
|1996|    1| 19|0.013043478563211966|
|2014|   12| 17| 0.02513661247784974|
|2005|   12| 25|0.017293233396415423|
+----+-----+---+--------------------+
only showing top 20 rows

root
 |-- year: integer (nullable = true)
 |-- month: integer 

In [74]:
T2.createOrReplaceTempView("T2_table")

In [76]:
Q="SELECT year,month,AVG(avg_daily) AS avg_monthly_precip FROM T2_table GROUP BY year,month ORDER BY year ASC,month ASC"
spark.sql(Q).show()

+----+-----+--------------------+
|year|month|  avg_monthly_precip|
+----+-----+--------------------+
|1993|    4|                 0.0|
|1993|    5|0.028360215105837392|
|1993|    6| 0.08851506861297961|
|1993|    7|  0.1282258073208473|
|1993|    8| 0.13591977530813945|
|1993|    9| 0.05759920684679869|
|1993|   10|0.058152174357674905|
|1993|   11| 0.05952020228624042|
|1993|   12| 0.04989481123243538|
|1994|    1| 0.04071698705971345|
|1994|    2|0.033709180670613854|
|1994|    3| 0.05174230323386763|
|1994|    4| 0.03295634947273703|
|1994|    5| 0.03373655926195845|
|1994|    6| 0.06393115995038705|
|1994|    7|                 0.0|
|1994|    8| 0.08052155520122158|
|1994|    9| 0.13360988390269843|
|1994|   10| 0.04476390831005273|
|1994|   11|0.021805555714915194|
+----+-----+--------------------+
only showing top 20 rows

